# Evenements culturels : Création d'une carte interactive

Durant ce projet, nous allons utiliser la base de données répertoriant les évènements culturels en île de France (https://data.iledefrance.fr/explore/dataset/evenements-publics-cibul/table/) pour extraire les différents évènements et récupérer les informations importantes comme la date, le prix, ou le sujet et les mettre sous des formes faciles à utiliser.

Tout d'abord, nous allons import le dataframe et récupérer les informations que nous pouvons principalement grâce à des expressions régulières et à du web scraping. Ensuite, pour obtenir le sujet des évènements nous allons utiliser du NLP en trouvant les termes revenant le plus souvent. Finalement, nous finirons par analyser les données trouvées.

## Installations et imports de modules

In [229]:
# Trivia
import pandas as pd
import numpy as np
from collections import Counter

# Regex
import re

# API, data import
import urllib
import bs4
from urllib import request
import requests

# User informations
import sys
!{sys.executable} -m pip install geopy
import geopy
from geopy.geocoders import Nominatim
from geopy import distance
from datetime import date
from datetime import timedelta

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt
import base64
import string
import re

# Natural Language Processing
!pip install nltk
import nltk
from nltk.corpus import stopwords
from time import time
#from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stopwords
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
!pip install pywaffle
from pywaffle import Waffle
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('french'))

!pip install spacy
!pip install https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.1.0/fr_core_news_sm-3.1.0.tar.gz

import spacy

nlp = spacy.load("fr_core_news_sm")

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


     |████████████████████████████████| 17.1 MB 174 kB/s            
  Preparing metadata (setup.py) ... done


/opt/conda/lib/python3.9/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'fr_core_news_sm' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.2.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


## Données : Collecte et Nettoyage

In [230]:
class User:
    ''' The User Class allows us to store every piece of information about the User '''
    
    def __init__(self, address, date, price_max = 42):
        geolocator = Nominatim(user_agent="CulturalEvents")
        location = geolocator.geocode(address)
        self.latlon = (location.latitude, location.longitude)
        self.date = date
        self.price_max = price_max
        
Naclanga = User("27 Boulevard Thomas Gobert, Palaiseau", date.today())

In [231]:
# Query to extract a specific frame of time, the user desired date
query = "date_start < " + str(Naclanga.date + timedelta(days=7)) + " AND date_start > "  + str(Naclanga.date)

# Request. We extract 100 rows, it can be changed up to 1000 rows
url = "https://data.iledefrance.fr/api/records/1.0/search/?dataset=evenements-publics-cibul&q=&rows=100&q=(" + query + ")&sort=date_start"
req = requests.get(url)
df = pd.json_normalize(req.json()['records'])

# Rename the columns
important_keys = ["fields.title", "fields.latlon", "fields.date_start", "fields.tags", "fields.pricing_info", "fields.description",'fields.link', 'fields.free_text', 'fields.address', 'fields.department','fields.placename', 'fields.region', 'fields.date_end', 'fields.lang', 'geometry.type', 'geometry.coordinates', 'fields.city_district']
df = df[important_keys]
dict_important_keys = {elt : elt.replace("fields.", "") for elt in important_keys}
df.rename(columns = dict_important_keys, inplace=True)

df_to_treat = df.copy()
df_to_treat

,title,latlon,date_start,tags,pricing_info,description,link,free_text,address,department,placename,region,date_end,lang,geometry.type,geometry.coordinates,city_district
0,Où va le monde ? les possibles scénarios futur...,"[48.800506, 2.131408]",2022-01-10,NaN,Sur inscription - Tarif B (Cycle de 8 conféren...,Royaume-Uni ou Royaume-Désuni ?,http://openagenda.com/event/ou-va-le-monde-les...,"PAR MANLIO GRAZIANO, ENSEIGNANT À SCIENCES PO ...","6 impasse des Gendarmes, entrée B 78000 Versai...",Yvelines,Université Ouverte de Versailles,Île-de-France,2022-01-10,fr,Point,"[2.131408, 48.800506]",Chantiers
1,Vie professionnelle,"[48.8558, 2.355934]",2022-01-10,NaN,NaN,Réunion d'accueil et d'information,http://openagenda.com/event/travail-et-chomage...,"La Fraternité ""Vie professionnelle"" accompagne...","Rue du Grenier sur l'eau, 75004 Paris",Paris,Salle Béthanie,Île-de-France,2022-12-05,fr,Point,"[2.355934, 48.8558]",NaN
2,"Patines à boire, patines à manger","[48.856697, 2.351462]",2022-01-10,"formations art,couleurs,pigment,patines sur bo...",renseignements( laisser vos coordonnées téléph...,Recettes d'antan et secrets d'ateliers,http://openagenda.com/event/atelier-patines-a-...,"Nous découvrirons ensemble, différentes patine...",Paris,Paris,Atelier Décodalice,Île-de-France,2022-01-12,fr,Point,"[2.351462, 48.856697]",20 eme
3,L’HOMME VIT D’OFFRANDE : QUE SUIS JE PRÊT À OF...,"[48.84834, 2.351983]",2022-01-10,NaN,NaN,Les réalités essentielles.,http://openagenda.com/event/lhomme-vit-doffran...,Informations\r\n\r\nhttps://www.collegedesbern...,"20 Rue de Poissy, 75005 Paris, France",Paris,Collège des Bernardins,Île-de-France,2022-01-10,fr,Point,"[2.351983, 48.84834]",5e Arrondissement
4,Regards croisés – ϕχψ « connaître »,"[48.851428, 2.295507]",2022-01-10,NaN,NaN,"Sur des questions de fond, nous réfléchirons a...",http://openagenda.com/event/regards-croises-xp...,"Des soirées indépendantes les unes des autres,...","11 place Dupleix, 75015 Paris",Paris,Salle Saint-Augustin (Maison des Œuvres),Île-de-France,2022-01-10,fr,Point,"[2.295507, 48.851428]",Paris 15e Arrondissement
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,I MUVRINI,"[48.700418, 2.467514]",2022-01-07,"chanson,I MUVRINI,MONTGERON,ESSONNE,ILE DE FRANCE",Tarifs: de 42€ à 37€,I MUVRINI CHANTENT UNE CORSE INATTENDUE. ILS C...,http://openagenda.com/event/i-muvrini-9032479,Non les lucioles n’ont pas disparu. Elles ne d...,"121 Avenue de la république, 91230 Montgeron",Essonne,L'ASTRAL,Île-de-France,2022-01-07,fr,Point,"[2.467514, 48.700418]",NaN
96,La collection de céramique attique du musée Pa...,"[48.866484, 2.338931]",2022-01-07,Vases grecs,Entrée libre dans la limite des places disponi...,Séminaire,http://openagenda.com/event/la-collection-de-c...,Le musée Paul et Alexandra Canellopoulos se tr...,"2, rue Vivienne ou 6 rue des Petits Champs",Paris,Institut national d'histoire de l'art - Galeri...,Île-de-France,2022-01-07,fr,Point,"[2.338931, 48.866484]",Paris 2e Arrondissement
97,JE VOLE ET LE RESTE JE LE DIRAI AUX OMBRES,"[48.823163, 1.95804]",2022-01-07,NaN,"A partir de 11,00€",THEÂTRE,http://openagenda.com/event/je-vole-et-le-rest...,"BANDE ANNONCE\n\n\n\t\n\t\t\n\t\t\t\n\t\t\t"" O...",980 AVENUE DU GENERAL DE GAULLE,NaN,Espace Coluche,NaN,2022-01-07,fr,Point,"[1.95804, 48.823163]",NaN
98,Les démons familiers,"[48.851104, 2.458721]",2022-01-07,"comptoir,roublot,fontenay-sous-bois,concert,li...",Plein tarif : 16 € / Tarif réduit : 12 € (chôm...,Résidence / Création,http://openagenda.com/event/les-demons-familiers,Avec ces « démons familiers » ouverts à l’alté...,Halle Roublot 95 rue Roublot 94120 Fontenay-so...,Val-de-Marne,Le Comptoir,Île-de-France,2022-01-07,fr,Point,"[2.458721, 48.851104]","Rigollots, Roublot, Carrières"


### Calcul de la distance

Grâce aux informations contenues dans le tableau et celles fournies par l'utilisateur. On peut calculer la distance géographique entre l'évènement et l'adresse de l'utilisateur

In [250]:
df_to_treat['distance'] = df['latlon'].apply(lambda x: distance.distance(x, Naclanga.latlon))
df_to_treat['distance']

0     10.917727275068321 km
1     19.645031498272726 km
2     19.534942631112997 km
3      18.80561872963572 km
4      16.96397759443594 km
              ...          
95    19.839138714207593 km
96    19.943270224380235 km
97     21.51660366392793 km
98     24.52210951009868 km
99    23.394745297709637 km
Name: distance, Length: 100, dtype: object

### Extraction du prix

Dans ce second cas les informations sont dans le tableau mais pas sous une forme très utile. Par conséquent on utilise les expressions régulières pour obtenir des prix en nombre

In [234]:
def extract_price_regex(x):
    ''' Returns an estimation of the price of the event '''
    
    # Multiple price can be there so we are going to find them all and compute the mean
    L = []
    for k in re.findall("\S+\s?€", str(x)):
        try:
            a = float(k.replace("€", "").replace(",", ".")) # Conversion

        # If there is no price or it doesn't work
        except ValueError:
            a = float("NaN")
        
        L.append(a)
        
    return np.mean(L)

df_to_treat['pricing_info'] = df['pricing_info'].apply(lambda x:extract_price_regex(x))

In [249]:
df_to_treat['pricing_info']

0     108.18
1        NaN
2        NaN
3        NaN
4        NaN
       ...  
95     39.50
96       NaN
97     11.00
98       NaN
99      5.00
Name: pricing_info, Length: 100, dtype: float64

### Extraction des horaires

Dans le troisième cas, l'information n'est pas dans le tableau mais on peut la trouver grâce au lien associé, d'où l'utilisation de webscraping.

In [253]:
def get_text(link):
    ''' Requests the html code of the page '''
    request_text = request.urlopen(link).read()
    page = bs4.BeautifulSoup(request_text)
    return(page)

def get_hours_regex(text):
    ''' Returns the hours of opening on the html page '''
    return str(text.select('span.hour')[0]).replace('<span class="hour">', "").replace("</span>", "")
    
def get_timespan(link):
    ''' Given a link, this function uses the last two to return the hours of opening '''

    try:
        text = get_text(link)
        hours = get_hours_regex(text)

    except (urllib.error.HTTPError, IndexError): # If the Event is not correctly setup (error 404) or the event is not accessible (error 403)
        hours = float("NaN")
        
    return hours

# Time consuming, so we should get all information at once to avoid doing too much webscraping
df_to_treat['opening_hours'] = df['link'].apply(lambda link: get_timespan(link))

# Should we differency ponctual events (movies, plays, sports) and long running events (expositions) ?
# -> We can probably do it judging the timespan of the events in terms of days and hours, but is it useful ?

In [254]:
df_to_treat['opening_hours']

0     14:30 - 16:00
1     19:00 - 21:00
2     09:30 - 12:30
3     20:00 - 21:30
4     20:30 - 22:00
          ...      
95    20:30 - 22:00
96    14:00 - 16:00
97    20:30 - 22:30
98    20:45 - 22:30
99    20:30 - 23:00
Name: opening_hours, Length: 100, dtype: object

## Natural Language Processing

### Création de Labels

Nous allons chercher les mots les plus utilisés pour créer des labels. Ici, on utilise toute la base de données sans faire de requête pour avoir le maximum de données

In [239]:
database = pd.read_csv('events_database.zip', compression='zip', header=0, sep=';')

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [240]:
database[['Description', 'Détails', 'Mots clés']].head(5)

,Description,Détails,Mots clés
0,Découvrir le poney dans un séjour multisports ...,L'UCPA propose ses vacances apprenantes : Un s...,"UCPA,sport,nature,SFJBOI31E,import-ucpa-20200630"
1,"Un séjour qui combine sport, collectivité et f...",Cette colo te permettra de découvrir les multi...,"jeux olympiques,sport,import-pep"
2,Le top de la glisse de la côte californienne,L'UCPA propose ses vacances apprenantes : Un s...,"UCPA,sport,nature,SFJPON34R,import-ucpa-20200630"
3,Compagnie AtmosphèreS - mise en scène : Sara V...,"Camille, Camille, Camille\n-------------------...","Camille Claudel,Rodin,Paul claudel,sculptrice,..."
4,Mise en scène par Agnès Braunschweig. Cie Pros...,Je reviens de la vérité de Charlotte Delbo. Mi...,"Résistance,Déportation,Deuxième guerre mondial..."


In [255]:
def NaN_to_String(dataframe):
    ''' Convert the NaN value to Empty String and returns a cleaner text '''
    df_temp = dataframe.copy()
    df_temp[df_temp.isna()] = ""
    return df_temp.apply(lambda s: s.replace("\n", "").replace("\r", "") + ". ")

# Variable containing every piece of possibly useful text in the database
text_desc = (NaN_to_String(database['Description']) + NaN_to_String(database['Détails']) + NaN_to_String(database['Mots clés']).apply(lambda s: s.replace(",", ", ")))
text_desc

0         Découvrir le poney dans un séjour multisports ...
1         Un séjour qui combine sport, collectivité et f...
2         Le top de la glisse de la côte californienne. ...
3         Compagnie AtmosphèreS - mise en scène : Sara V...
4         Mise en scène par Agnès Braunschweig. Cie Pros...
                                ...                        
142884    Théâtre - Conférence / À partir de 14 ans   Pe...
142885    J'AI DES DOUTES. Molière 2019 du Comédien dans...
142886    Jazz vocal      Une formation alliant l’esprit...
142887    Théâtre   « Femme, réveille-toi » disait Olymp...
142888    CATCH D'IMPRO Encadrés par un arbitre qui s’oc...
Length: 142889, dtype: object

Premier test

On cherche les mots les plus utilisés grâce au module nltk en enlevant les mots qui ne sont pas composés de caractère alphabétique et les mots les plus usités en français (stop_words : le, la ...)

In [258]:
text = text_desc.head(100).sum().lower() # We use only 1000 rows but we can go higher, it just take more time
words = nltk.word_tokenize(text, language='french')
words = [word for word in words if word.isalpha()]
words = [w for w in words if not w in stop_words]
dic_words = Counter(words)

In [259]:
dic_words.most_common(20)

[('a', 4404),
 ('plus', 3203),
 ('paris', 2476),
 ('musique', 2439),
 ('cette', 2386),
 ('https', 2328),
 ('the', 2253),
 ('http', 1935),
 ('tout', 1624),
 ('entre', 1623),
 ('comme', 1473),
 ('scène', 1455),
 ('concert', 1438),
 ('monde', 1387),
 ('amp', 1378),
 ('ans', 1351),
 ('tous', 1351),
 ('public', 1306),
 ('théâtre', 1305),
 ('découvrir', 1288)]

Comme on peut le voir beaucoup de mots ne correspondent pas à nos attentes, certains ne sont même pas des mots d'ailleurs.

Une idée serait d'utiliser une méthode d'étiquettage morpho syntaxique, ainsi on pourrait conserver seulement les noms communs. Pour cela il faut utiliser spacy plutôt que nltk car le français n'est pas supporté pour cette méthode dans nltk.

Second Test : L'étiquettage morpho-syntaxique

In [244]:
def return_POS(sentence, n = -1):
    ''' Returns a list of tags, there are the most used nouns in the database. n is the number of tags that we want. If n = -1, every possible tags are returned '''
    
    # Tokenize
    doc = nlp(sentence)
    
    # We only keep words being labellized as nouns
    words = [[X.lemma_, X.lemma_] for X in doc if (X.is_alpha == True and X.pos_ == "NOUN" and not(X in stop_words))]
    df_words = pd.DataFrame(words, columns=['tags', 'unique_tags'])
    
    if n == -1:
        return list(df_words.groupby(['unique_tags']).count().sort_values('tags', ascending=False).to_dict()['tags'].keys())
    else:
        return list(df_words.groupby(['unique_tags']).count().sort_values('tags', ascending=False).head(n).to_dict()['tags'].keys())

In [245]:
tags = return_POS(text_desc.head(1000).sum().lower(), 50)
tags

['musique',
 'atelier',
 'théâtre',
 'concert',
 'projet',
 'visite',
 'scène',
 'monde',
 'histoire',
 'année',
 'groupe',
 'samedi',
 'exposition',
 'jeu',
 'public',
 'enfant',
 'spectacle',
 'an',
 'place',
 'dimanche',
 'création',
 'rencontre',
 'conférence',
 'recherche',
 'association',
 'vie',
 'aide',
 'travers',
 'cadre',
 'découverte',
 'festival',
 'temps',
 'artiste',
 'maison',
 'formation',
 'œuvre',
 'danse',
 'siècle',
 'musée',
 'travail',
 'ville',
 'soirée',
 'journée',
 'heure',
 'université',
 'occasion',
 'programme',
 'chanson',
 'centre',
 'rue']

In [246]:
df_to_treat['text'] = (NaN_to_String(df['tags']) + NaN_to_String(df['title']) + NaN_to_String(df['description']) + NaN_to_String(df['free_text'])).apply(lambda s: s.replace(",", ", "))
df_to_treat['tags'] = df_to_treat['text'].apply(lambda x:return_POS(x.lower()))
df_to_treat['tags'] = df_to_treat['tags'].apply(lambda x: [k for k in x if k in tags])

In [247]:
df_to_treat['tags'] 

0                                   [monde, université]
1                                 [vie, travail, monde]
2                           [formation, atelier, année]
3                                                    []
4                                              [soirée]
                            ...                        
95                          [spectacle, monde, chanson]
96    [œuvre, histoire, recherche, musée, monde, siè...
97    [scène, histoire, rencontre, spectacle, vie, t...
98                 [création, projet, musique, concert]
99                             [création, concert, rue]
Name: tags, Length: 100, dtype: object

In [248]:
df_to_treat[df_to_treat['tags'].apply(lambda x:x==[])]

,title,latlon,date_start,tags,pricing_info,description,link,free_text,address,department,placename,region,date_end,lang,geometry.type,geometry.coordinates,city_district,distance,opening_hours,text
3,L’HOMME VIT D’OFFRANDE : QUE SUIS JE PRÊT À OF...,"[48.84834, 2.351983]",2022-01-10,[],NaN,Les réalités essentielles.,http://openagenda.com/event/lhomme-vit-doffran...,Informations\r\n\r\nhttps://www.collegedesbern...,"20 Rue de Poissy, 75005 Paris, France",Paris,Collège des Bernardins,Île-de-France,2022-01-10,fr,Point,"[2.351983, 48.84834]",5e Arrondissement,18.80561872963572 km,20:00 - 21:30,. L’HOMME VIT D’OFFRANDE : QUE SUIS JE PRÊT À ...
6,Club de lecture,"[48.829127, 2.317717]",2022-01-10,[],NaN,"Pour tous les passionnés de littérature, les d...",http://openagenda.com/event/club-de-lecture-97...,"Pour tous les passionnés de littérature, les d...",51 rue de l'Abbé Carton 75014 Paris,Paris,La Table des Matières,Île-de-France,2022-06-13,fr,Point,"[2.317717, 48.829127]",Paris 14e Arrondissement,15.604271058210436 km,19:00 - 20:30,"lecture, club, littérature. Club de lecture. P..."
18,Franco Fagioli : Mozart et Les Castrats,"[48.804425, 2.120285]",2022-01-09,[],NaN,Mozart et les Castrats,http://openagenda.com/event/franco-fagioli-moz...,Si la passion de Mozart pour les voix féminine...,Château de Versailles,Yvelines,Opéra Royal,Île-de-France,2022-01-09,fr,Point,"[2.120285, 48.804425]",Notre-Dame,11.688576595126854 km,15:00 - 17:00,. Franco Fagioli : Mozart et Les Castrats. Moz...
25,NaN,"[48.529633, 2.292857]",2022-01-09,[],NaN,NaN,http://openagenda.com/event/les-itinerantes-co...,NaN,rue René Cassin 91510 Lardy,Essonne,"Salle Cassin,",Île-de-France,2022-01-09,fr,Point,"[2.292857, 48.529633]",NaN,21.53819397347843 km,16:00 - 17:30,. . . .
37,Hl. Drei Königsgottesdienst,"[48.868801, 2.277584]",2022-01-09,[],NaN,Familiengottesdienst mit Sternsingern,http://openagenda.com/event/hl-drei-konigsgott...,NaN,"38 rue Spontini, Paris 16ème",Paris,Katholische Gemeinde Sankt Albertus Magnus,Île-de-France,2022-01-09,fr,Point,"[2.277584, 48.868801]",NaN,18.274952812524763 km,11:00 - 12:00,. Hl. Drei Königsgottesdienst. Familiengottesd...
38,Messe des familles,"[48.845708, 2.403524]",2022-01-09,[],NaN,Suivie de la première réunion de préparation à...,http://openagenda.com/event/messe-des-familles...,NaN,34 rue du Rendez-vous 75012 PARIS,Paris,Eglise de l'Immaculée Conception,Île-de-France,2022-01-09,fr,Point,"[2.403524, 48.845708]",NaN,21.087085780669245 km,10:00 - 11:00,. Messe des familles. Suivie de la première ré...
40,EK Katechese 4,"[48.868801, 2.277584]",2022-01-08,[],NaN,Viertes Treffen zur Vorbereitung auf die Erstk...,http://openagenda.com/event/ek-katechese-4,NaN,"38 rue Spontini, Paris 16ème",Paris,Katholische Gemeinde Sankt Albertus Magnus,Île-de-France,2022-01-08,fr,Point,"[2.277584, 48.868801]",NaN,18.274952812524763 km,14:30 - 17:00,. EK Katechese 4. Viertes Treffen zur Vorberei...
57,Stage d'improvisation théâtrale,"[48.890801, 2.375054]",2022-01-08,[],NaN,Stage Impro pour adulte,http://openagenda.com/event/stage-dimprovisati...,NaN,"15 rue Mathis, 75019",Paris,Centre Paris Anim'Mathis,Île-de-France,2022-01-08,fr,Point,"[2.375054, 48.890801]",NaN,23.64469103145909 km,15:30 - 18:00,. Stage d'improvisation théâtrale. Stage Impro...
61,LE LANGAGE DES CORPS : LA DIFFÉRENCE SEXUELLE,"[48.84834, 2.351983]",2022-01-08,[],NaN,Le corps et son mystère - une anthropologie sp...,http://openagenda.com/event/le-langage-des-cor...,Informations\r\n\r\nhttps://www.collegedesbern...,"20 Rue de Poissy, 75005 Paris, France",Paris,Collège des Bernardins,Île-de-France,2022-01-08,fr,Point,"[2.351983, 48.84834]",5e Arrondissement,18.80561872963572 km,10:30 - 12:00,. LE LANGAGE DES CORPS : LA DIFFÉRENCE SEXUELL...
63,NaN,"[48.893501, 2.4572]",2022-01-08,[],NaN,NaN,http://openagenda.com/event/tapis-tipis-1227947,NaN,"3 Rue Jean Jaurès, 93130 Noisy-le-Sec",NaN,Médiathèque Roger Gouhier - Noisy-le-Sec,NaN,2022-01-08,fr,Point,"[2.4572, 48.893501]",NaN,27.63